# costPower

In [28]:
import geocoder

In [4]:
import requests
import pymongo
import geocoder
import time
#/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/8/1124193D-09F5-4711-AB9A-01848E3B88E4/8/?fileType=csv
res=requests.get("https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/8/1124193D-09F5-4711-AB9A-01848E3B88E4/8/?fileType=csv")
x=res.text
keys=x.replace('"',"").replace("\u3000","").replace("\U000fffb4","").replace("\r","").split("\n")[0].split(",")
values=[data.replace("\u3000","")
        .replace("\U000fffb4","")
        .replace("\U000fb56f","塭")
        .replace("\r","")
        .split(",") for data in x.replace('"',"").split("\n")[1:]]
datalist=[]
for valueList in values:
    data={}
    for key,value in zip(keys,valueList):
        data[key]=value
    datalist.append(data)
dataFinal=[]
for i in datalist:
    data={}
    if i.get('主行業別',"")=='餐飲業' and i.get('年度',"")=='2017' and i.get("消費熱度計算來源","")=="電子發票":
        data['add']=i['縣市']+i['鄉鎮市區']+i['村里']
        data['bigadd']=i['縣市'].replace("臺","台")
        data['smalladd']=i['鄉鎮市區']
        data['costPower']=i['銷售額指標']
        data['invoice']=i['張數指標']
#         try:
#             data['lat'],data['lng']=geocoder.google(data['add']).latlng
#         except:
#             time.sleep(2.5)
#             try:
#                 data['lat'],data['lng']=geocoder.google(data['add']).latlng
#             except:
#                 time.sleep(2.5)
#                 data['lat'],data['lng']=geocoder.google(data['add']).latlng
#         time.sleep(2.5)
        dataFinal.append(data)
#         print(data['add'],data['lat'],data['lng'])
        
        
        
client=pymongo.mongo_client.MongoClient("192.168.1.113",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in dataFinal:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
costPower=[add for add in dataFinal if 'lat' in add]
client=pymongo.mongo_client.MongoClient("192.168.1.113",27017,username='j122085',password='850605')
db=client.rawData
collection=db.CostPower
try:
    collection.drop()
    collection.insert_many(costPower,ordered=False)
except:
    print("some data has same _id")
    
print("xxx")
client.close()

['MRTinfo', 'info3Store', 'websites591', 'taiwanInfo', 'trainStation', 'eslite', 'ipeenInfo', 'conStore', 'CostPower', 'infoClinic', 'Watsons', 'pxmart', 'busData', 'HRdata104', 'wowprimediendata', 'info591', 'departmentStore', 'carrefour', 'smallStyleCount', 'addressCoordinate', 'Nhuman', 'taiwanInfoStoneTwo', 'ipeenWebsite', 'realPriceLogin', 'taiwanInfoHot7', 'bigStyleCount', 'taiwanInfoMaBoss']
xxx


# humanDensity

In [5]:
import requests
import json
import pymongo
import re
# res=requests.get("https://quality.data.gov.tw/dq_download_json.php?nid=32973&md5_url=52629fae1f1fed4ef81fcf706878dd80")
# x=json.loads(res.text)

res=requests.get("http://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=4C79B290-D72F-4DC9-A919-9023F141E32C")
datas=res.text.split("\r\n")
titles=datas[0].split(",")
x=[{title:value for title,value in zip(titles,data.split(","))} for data in datas[2:]]

data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五").replace("?","館"),
       'Nhuman':add['people_total']}for add in x[1:] if "site_id" in add]
for i in data:
    try:
        i["bigadd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                               i["add"])[0][0].replace("臺","台")
    except:
        i['bigadd']=0
    try:
        i["smalladd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                           i["add"])[0][1]
    except:
        i["smalladd"]=0
#     try:
#         data['lat'],data['lng']=geocoder.google(data['add']).latlng
#     except:
#         time.sleep(2.5)
#         try:
#             data['lat'],data['lng']=geocoder.google(data['add']).latlng
#         except:
#             time.sleep(2.5)
#             data['lat'],data['lng']=geocoder.google(data['add']).latlng
#     time.sleep(2.5)
    dataFinal.append(data)
#     print(data['add'],data['lat'],data['lng'])


        
client=pymongo.mongo_client.MongoClient("192.168.1.113",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in data:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
Nhuman=[add for add in data if 'lat' in add]
client=pymongo.mongo_client.MongoClient("192.168.1.113",27017,username='j122085',password='850605')
db=client.rawData
collection=db.Nhuman
try:
    collection.drop()
    collection.insert_many(Nhuman,ordered=False)
except:
    print("some data has same _id")
client.close()

['MRTinfo', 'info3Store', 'websites591', 'taiwanInfo', 'trainStation', 'eslite', 'ipeenInfo', 'conStore', 'CostPower', 'infoClinic', 'Watsons', 'pxmart', 'busData', 'HRdata104', 'wowprimediendata', 'info591', 'departmentStore', 'carrefour', 'smallStyleCount', 'addressCoordinate', 'Nhuman', 'taiwanInfoStoneTwo', 'ipeenWebsite', 'realPriceLogin', 'taiwanInfoHot7', 'bigStyleCount', 'taiwanInfoMaBoss']


# 轉換區里成經緯度

In [36]:
import requests
import geocoder
import time

res=requests.get("http://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=4C79B290-D72F-4DC9-A919-9023F141E32C")
datas=res.text.split("\r\n")
titles=datas[0].split(",")
x=[{title:value for title,value in zip(titles,data.split(","))} for data in datas[2:]]

data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五").replace("?","館"),
       'Nhuman':add['people_total']}for add in x[1:] if "site_id" in add]
addlist=[{"add":i['add']} for i in data]


for i in addlist:
    n=0
    while 'lat' not in i:
        try:
            n+=1
            if n>10:
                break
            i['lat'],i['lng']=geocoder.arcgis(i['add']).latlng
            time.sleep(0.5)
            print(i['add'],i['lat'],i['lng'])
        except:
            time.sleep(0.3)
        

for i in addlist:
    if "新竹市東區" in i['add'] or "新竹市北區" in i['add'] or "彰化縣彰化市" in i['add'] or "嘉義市東區" in i['add'] or "嘉義市西區" in i['add'] or "高雄市苓雅區" in i['add']:
        n=0
        try:
            n+=1
            if n>10:
                break
            i['lat'],i['lng']=geocoder.google(i['add']).latlng
            time.sleep(0.8)
            print(i['add'],i['lat'],i['lng'])
        except:
            time.sleep(0.5)
        

新北市板橋區流芳里 25.00855000000007 121.45719000000008
新北市板橋區赤松里 25.009250000000065 121.45663000000002
新北市板橋區黃石里 25.010800000000074 121.4572300000001
新北市板橋區挹秀里 25.00998000000004 121.45916000000011
新北市板橋區湳興里 25.00618000000003 121.45302000000004
新北市板橋區新興里 25.00660000000005 121.45552000000009
新北市板橋區社後里 25.015090000000043 121.4566000000001
新北市板橋區香社里 25.018890000000056 121.45135000000005
新北市板橋區中正里 25.026290000000074 121.45451000000003
新北市板橋區自強里 25.012740000000065 121.45522000000005
新北市板橋區自立里 25.014770000000055 121.45353000000011
新北市板橋區光華里 25.00976000000003 121.45329000000004
新北市板橋區國光里 25.017960000000073 121.45881000000008
新北市板橋區港尾里 25.02117000000004 121.45598000000007
新北市板橋區金華里 25.019810000000064 121.45859000000007
新北市板橋區港德里 25.019160000000056 121.45877000000007
新北市板橋區民權里 25.01484000000005 121.45971000000009
新北市板橋區建國里 25.01652000000007 121.45924000000002
新北市板橋區漢生里 25.01529000000005 121.46161000000006
新北市板橋區公舘里 25.018570000000068 121.46223000000009
新北市板橋區新民里 25.015450000000044 121.46428000000003
新北市

新北市三重區自強里 25.068020000000047 121.49468000000002
新北市三重區文化里 25.06424000000004 121.49932000000001
新北市三重區中央里 25.06720000000007 121.49775000000011
新北市三重區雙園里 25.068480000000022 121.49701000000005
新北市三重區錦通里 25.063640000000078 121.50115000000005
新北市三重區錦安里 25.065920000000062 121.4996000000001
新北市三重區長安里 25.068660000000023 121.49923000000001
新北市三重區光華里 25.065930000000037 121.50192000000004
新北市三重區長生里 25.06692000000004 121.50119000000007
新北市三重區開元里 25.065660000000037 121.50364000000002
新北市三重區大德里 25.064990000000023 121.50331000000006
新北市三重區長元里 25.06651000000005 121.50223000000005
新北市三重區長江里 25.067990000000066 121.5036500000001
新北市三重區長泰里 25.068850000000054 121.50165000000004
新北市三重區長福里 25.068980000000067 121.50390000000004
新北市三重區介壽里 25.069210000000055 121.50024000000008
新北市三重區萬壽里 25.069990000000075 121.50196000000005
新北市三重區錦江里 25.070660000000032 121.50301000000002
新北市三重區龍濱里 25.07299000000006 121.50228000000004
新北市三重區龍門里 25.07334000000003 121.49961000000008
新北市三重區秀江里 25.07236000000006 121.50024000000008
新

新北市永和區永元里 25.00139000000007 121.52087000000006
新北市永和區秀林里 25.00266000000005 121.52308000000005
新北市永和區保安里 25.011340000000075 121.50278000000003
新北市永和區中溪里 25.012260000000026 121.5099100000001
新北市永和區永安里 25.005400000000066 121.51526000000001
新北市永和區民族里 24.998350000000073 121.51933000000008
新北市永和區民權里 24.99792000000008 121.52069000000006
新北市永和區中興里 25.013190000000066 121.5177000000001
新北市永和區勵行里 25.01451000000003 121.51666000000012
新北市永和區秀朗里 25.003630000000044 121.52600000000007
新北市永和區秀成里 25.003580000000056 121.5276
新北市永和區上溪里 25.015100000000075 121.51459000000011
新北市永和區竹林里 25.010420000000067 121.52213000000006
新北市永和區雙和里 25.00117000000006 121.50813000000005
新北市永和區安和里 25.004020000000025 121.51084000000003
新北市永和區水源里 25.005720000000053 121.51083000000006
新北市永和區協和里 25.00789000000003 121.5121200000001
新北市永和區河堤里 25.018140000000074 121.51294000000007
新北市永和區頂溪里 25.01669000000004 121.51523000000009
新北市永和區潭墘里 25.00216000000006 121.51569000000006
新北市永和區潭安里 24.99875000000003 121.51568000000009
新北市永和區大新里 25.0

新北市新店區新德里 24.966710000000035 121.54088000000002
新北市新店區張北里 24.972200000000043 121.54530000000011
新北市新店區五峯里 24.967340000000036 121.54140000000007
新北市新店區新安里 24.977310000000045 121.54410000000007
新北市新店區忠誠里 24.975510000000043 121.54216000000008
新北市新店區中正里 24.97192000000007 121.53831000000002
新北市新店區中華里 24.969670000000065 121.5386400000001
新北市新店區仁愛里 24.974950000000035 121.53830000000005
新北市新店區百忍里 24.975410000000068 121.53596000000005
新北市新店區百和里 24.977850000000046 121.53534000000002
新北市新店區福德里 24.975940000000037 121.5328300000001
新北市新店區百福里 24.972200000000043 121.53506000000004
新北市新店區福民里 24.972780000000057 121.53101000000004
新北市新店區中山里 24.97738000000004 121.52688
新北市新店區和平里 24.98169000000007 121.52778
新北市新店區中央里 24.976480000000038 121.52520000000004
新北市新店區明城里 24.949510000000032 121.49882000000002
新北市新店區安和里 24.97457000000003 121.51739000000009
新北市新店區吉祥里 24.94572000000005 121.4953700000001
新北市新店區寶福里 24.971730000000036 121.55038000000002
新北市新店區小城里 24.946430000000078 121.49975000000006
新北市新店區香坡里 24.93487

新北市汐止區保長里 25.075000000000045 121.68353000000002
新北市汐止區鄉長里 25.085090000000037 121.66155000000003
新北市汐止區江北里 25.07466000000005 121.65637000000004
新北市汐止區拱北里 25.087600000000066 121.65056000000004
新北市汐止區烘內里 25.121150000000057 121.62983000000008
新北市汐止區八連里 25.098870000000034 121.62955000000011
新北市汐止區樟樹里 25.057060000000035 121.63397000000009
新北市汐止區北峰里 25.07349000000005 121.63155000000006
新北市汐止區北山里 25.070980000000077 121.62345000000005
新北市汐止區白雲里 25.042320000000075 121.65447000000006
新北市汐止區橫科里 25.047160000000076 121.62884000000008
新北市汐止區東山里 25.052170000000046 121.70438000000001
新北市汐止區福山里 25.043100000000038 121.62350000000004
新北市汐止區宜興里 25.039850000000058 121.62763000000007
新北市汐止區中興里 25.066900000000032 121.63387000000012
新北市汐止區湖光里 25.08355000000006 121.63639
新北市汐止區仁德里 25.07254000000006 121.66111000000001
新北市汐止區厚德里 25.06755000000004 121.64800000000002
新北市汐止區忠孝里 25.06260000000003 121.64453000000003
新北市汐止區自強里 25.058790000000045 121.65408000000002
新北市汐止區文化里 25.05695000000003 121.65286000000003
新北市汐止區金龍

...

屏東縣滿州鄉滿州村 22.02362000000005 120.83784000000003
屏東縣滿州鄉里德村 22.023690000000045 120.86112000000003
屏東縣滿州鄉永靖村 22.00932000000006 120.7991300000001
屏東縣滿州鄉港口村 21.99087000000003 120.82575000000008
屏東縣滿州鄉响林村 22.02072000000004 120.83881000000008
屏東縣滿州鄉長樂村 22.07462000000004 120.82882000000006
屏東縣滿州鄉九棚村 22.09890000000007 120.86705000000006
屏東縣滿州鄉港仔村 22.128390000000024 120.87301000000002
屏東縣枋山鄉枋山村 22.253940000000057 120.6611200000001
屏東縣枋山鄉加祿村 22.33114000000006 120.62645000000009
屏東縣枋山鄉楓港村 22.201410000000067 120.68869000000007
屏東縣枋山鄉善餘村 22.15217000000007 120.70044000000007
屏東縣三地門鄉三地村 22.73141000000004 120.66748000000007
屏東縣三地門鄉達來村 22.727690000000052 120.68029000000001
屏東縣三地門鄉德文村 22.802090000000078 120.72233000000006
屏東縣三地門鄉大社村 22.86239000000006 120.68402000000003
屏東縣三地門鄉賽嘉村 22.75001000000003 120.6518000000001
屏東縣三地門鄉口社村 22.771980000000042 120.65866000000005
屏東縣三地門鄉馬兒村 22.78630000000004 120.67178000000001
屏東縣三地門鄉安坡村 22.802730000000054 120.6432400000001
屏東縣三地門鄉青山村 22.820970000000045 120.67023000000006

花蓮縣花蓮市國威里 23.980560000000025 121.60612000000003
花蓮縣花蓮市國風里 23.979840000000024 121.59824000000003
花蓮縣花蓮市民德里 23.982420000000047 121.62207000000001
花蓮縣花蓮市民意里 24.00001000000003 121.61482000000001
花蓮縣花蓮市民治里 23.97234000000003 121.61203000000012
花蓮縣花蓮市主義里 23.97309000000007 121.60703000000001
花蓮縣花蓮市主工里 23.976750000000038 121.60341000000005
花蓮縣花蓮市主農里 23.969290000000058 121.59739000000002
花蓮縣花蓮市國治里 23.979170000000067 121.60817000000009
花蓮縣花蓮市國富里 23.991210000000024 121.59423000000004
花蓮縣花蓮市民政里 23.99649000000005 121.62402000000009
花蓮縣花蓮市民勤里 23.98912000000007 121.6173500000001
花蓮縣花蓮市民生里 23.97992000000005 121.61143000000004
花蓮縣花蓮市主商里 23.97580000000005 121.6083900000001
花蓮縣花蓮市主學里 23.97308000000004 121.60097000000007
花蓮縣花蓮市國光里 23.99088000000006 121.61044000000004
花蓮縣花蓮市國聯里 23.992630000000077 121.60298000000012
花蓮縣花蓮市國強里 24.00741000000005 121.60563000000002
花蓮縣花蓮市民心里 24.015450000000044 121.6286
花蓮縣花蓮市民樂里 23.988730000000032 121.63022000000001
花蓮縣花蓮市民主里 23.973960000000034 121.61363000000006
花蓮縣花蓮市主勤里 23.9

澎湖縣馬公市中央里 23.56352000000004 119.5642600000001
澎湖縣馬公市啟明里 23.56438000000003 119.56697000000008
澎湖縣馬公市重慶里 23.565900000000056 119.56769000000008
澎湖縣馬公市中興里 23.57006000000007 119.56432000000007
澎湖縣馬公市光復里 23.56800000000004 119.56941000000006
澎湖縣馬公市光榮里 23.57725000000005 119.57051000000001
澎湖縣馬公市光明里 23.57430000000005 119.56888000000004
澎湖縣馬公市朝陽里 23.573210000000074 119.57711000000006
澎湖縣馬公市陽明里 23.56910000000005 119.57530000000008
澎湖縣馬公市重光里 23.581840000000057 119.56865000000005
澎湖縣馬公市西衛里 23.575410000000034 119.57913000000008
澎湖縣馬公市西文里 23.569780000000037 119.58467000000007
澎湖縣馬公市東文里 23.561140000000023 119.58492000000001
澎湖縣馬公市案山里 23.553920000000062 119.57666000000006
澎湖縣馬公市光華里 23.552640000000054 119.56955000000005
澎湖縣馬公市前寮里 23.55501000000004 119.59091000000001
澎湖縣馬公市石泉里 23.55609000000004 119.5945200000001
澎湖縣馬公市菜園里 23.554670000000044 119.60111000000006
澎湖縣馬公市東衛里 23.564010000000053 119.6046500000001
澎湖縣馬公市安宅里 23.579480000000046 119.60469000000012
澎湖縣馬公市烏崁里 23.54800000000006 119.62186000000008
澎湖縣馬公

基隆市仁愛區獅球里 25.12447000000003 121.73720000000003
基隆市仁愛區書院里 25.124330000000043 121.73649000000012
基隆市仁愛區朝棟里 25.126550000000066 121.73628000000008
基隆市仁愛區明德里 25.128930000000025 121.73852000000011
基隆市仁愛區同風里 25.12814000000003 121.73948000000007
基隆市仁愛區文昌里 25.133440000000064 121.74040000000002
基隆市仁愛區新店里 25.129850000000033 121.74052000000006
基隆市仁愛區光華里 25.126840000000072 121.7373500000001
基隆市中山區新建里 25.130330000000072 121.73611000000005
基隆市中山區安民里 25.13086000000004 121.73687000000007
基隆市中山區安平里 25.132210000000043 121.7382
基隆市中山區中山里 25.13262000000003 121.73846000000003
基隆市中山區民治里 25.134390000000053 121.73962000000006
基隆市中山區中興里 25.138720000000035 121.73718000000008
基隆市中山區仁正里 25.14110000000005 121.73738000000003
基隆市中山區健民里 25.142000000000053 121.73704000000009
基隆市中山區通化里 25.146220000000028 121.7398300000001
基隆市中山區居仁里 25.143750000000068 121.73427000000004
基隆市中山區通明里 25.14722000000006 121.73864000000003
基隆市中山區協和里 25.157980000000066 121.73386000000005
基隆市中山區文化里 25.153580000000034 121.73448000000008
基隆市中山區仙洞里 

新竹市香山區虎山里 24.81212000000005 120.91632000000004
新竹市香山區虎林里 24.808710000000076 120.93376
新竹市香山區浸水里 24.801930000000027 120.91187000000002
新竹市香山區樹下里 24.79863000000006 120.93047000000001
新竹市香山區埔前里 24.796500000000037 120.93693000000007
新竹市香山區中埔里 24.796280000000024 120.94430000000011
新竹市香山區牛埔里 24.78812000000005 120.9389900000001
新竹市香山區頂埔里 24.79645000000005 120.95222000000001
新竹市香山區東香里 24.772070000000042 120.95263000000011
新竹市香山區香村里 24.77308000000005 120.93787000000009
新竹市香山區香山里 24.785840000000064 120.92301000000009
新竹市香山區大庄里 24.794800000000066 120.91666000000009
新竹市香山區美山里 24.781670000000076 120.91496000000006
新竹市香山區朝山里 24.765690000000063 120.90337000000011
新竹市香山區海山里 24.761010000000056 120.91813000000002
新竹市香山區茄苳里 24.754710000000046 120.95388000000003
新竹市香山區大湖里 24.745640000000037 120.9320100000001
新竹市香山區南隘里 24.729810000000043 120.93342000000007
新竹市香山區中隘里 24.72625000000005 120.91833000000008
新竹市香山區內湖里 24.744260000000054 120.91619000000003
新竹市香山區鹽水里 24.751040000000046 120.90528000000006
新竹市香山區南港里

In [6]:
# from pymongo import UpdateOne
# import pymongo
# client=pymongo.mongo_client.MongoClient("192.168.1.113",27017,username='j122085',password='850605')
# # print(client.database_names())
# db=client.rawData
# print(db.collection_names())
# collection=db.addressCoordinate
# addlist=list(collection.find({},{"_id":False}))


['MRTinfo', 'info3Store', 'websites591', 'taiwanInfo', 'trainStation', 'eslite', 'ipeenInfo', 'conStore', 'CostPower', 'infoClinic', 'Watsons', 'pxmart', 'busData', 'HRdata104', 'wowprimediendata', 'info591', 'departmentStore', 'carrefour', 'smallStyleCount', 'addressCoordinate', 'Nhuman', 'taiwanInfoStoneTwo', 'ipeenWebsite', 'realPriceLogin', 'taiwanInfoHot7', 'bigStyleCount', 'taiwanInfoMaBoss']


In [73]:
# for i in addlist:
#     if "新竹市東區" in i['add'] or "新竹市北區" in i['add'] or "彰化縣彰化市" in i['add'] or "嘉義市東區" in i['add'] or "嘉義市西區" in i['add'] or "高雄市苓雅區" in i['add']:
#         n=0
#         try:
#             n+=1
#             if n>10:
#                 break
#             i['lat'],i['lng']=geocoder.google(i['add']).latlng
#             time.sleep(0.8)
#             print(i['add'],i['lat'],i['lng'])
#         except:
#             time.sleep(0.5)

高雄市苓雅區博仁里 22.6200876 120.2927338
高雄市苓雅區苓洲里 22.6185963 120.2934547
高雄市苓雅區苓昇里 22.6169555 120.2948966
高雄市苓雅區苓中里 22.6150924 120.2941757
高雄市苓雅區苓雅里 22.6153147 120.2963385
高雄市苓雅區苓東里 22.6125201 120.2952571
高雄市苓雅區城北里 22.6135244 120.2985012
高雄市苓雅區城西里 22.6144176 120.300664
高雄市苓雅區城東里 22.61505 120.3024662
高雄市苓雅區意誠里 22.6122555 120.3013849
高雄市苓雅區鼓中里 22.6162079 120.2985012
高雄市苓雅區人和里 22.618293 120.3013849
高雄市苓雅區仁政里 22.6187758 120.303908
高雄市苓雅區廣澤里 22.6157934 120.3053498
高雄市苓雅區美田里 22.6206001 120.3064311
高雄市苓雅區華堂里 22.6179169 120.3064311
高雄市苓雅區日中里 22.6173185 120.3093145
高雄市苓雅區普照里 22.6183223 120.3125583
高雄市苓雅區和煦里 22.6206725 120.3093145
高雄市苓雅區晴朗里 22.6213047 120.3111166
高雄市苓雅區普天里 22.6190654 120.3154416
高雄市苓雅區林富里 22.6220477 120.314
高雄市苓雅區林圍里 22.6244703 120.3136396
高雄市苓雅區林安里 22.6259615 120.3129187
高雄市苓雅區光華里 22.6160443 120.3186853
高雄市苓雅區林興里 22.617907 120.3194061
高雄市苓雅區林華里 22.6165267 120.3212081
高雄市苓雅區林西里 22.6197696 120.3201269
高雄市苓雅區林中里 22.6214104 120.3186853
高雄市苓雅區林泉里 22.6226216 120.3185051
高雄市苓雅區林南里 22.6235723

嘉義市東區文雅里 23.4826518 120.485444
嘉義市東區安業里 23.4618443 120.4544857
嘉義市東區義教里 23.4975023 120.4537656
嘉義市西區香湖里 23.4958554 120.4458448
嘉義市西區湖邊里 23.4916303 120.4436845
嘉義市西區北榮里 23.4879194 120.446925
嘉義市西區重興里 23.4868897 120.440804
嘉義市西區竹圍里 23.4849319 120.4343228
嘉義市西區新厝里 23.4922491 120.440804
嘉義市西區保安里 23.4900856 120.4321623
嘉義市西區北湖里 23.4944152 120.4213592
嘉義市西區下埤里 23.4915282 120.409835
嘉義市西區竹村里 23.4983305 120.396869
嘉義市西區大溪里 23.4820428 120.4040725
嘉義市西區福全里 23.4790568 120.4336026
嘉義市西區西平里 23.4779227 120.4264007
嘉義市西區磚󿾨里 23.472769 120.4285613
嘉義市西區港坪里 23.4752414 120.4170377
嘉義市西區頭港里 23.4713209 120.4040725
嘉義市西區劉厝里 23.4665829 120.4199187
嘉義市西區新西里 23.4717387 120.4364832
嘉義市西區書院里 23.4772013 120.4422443
嘉義市西區培元里 23.4715326 120.4436845
嘉義市西區垂楊里 23.4711204 120.4393638
嘉義市西區車店里 23.4669972 120.4336026
嘉義市西區福民里 23.4626683 120.4350429
嘉義市西區湖內里 23.45195 120.4444046
嘉義市西區美源里 23.4663793 120.4458448
嘉義市西區育英里 23.4706048 120.4480051
嘉義市西區致遠里 23.4694712 120.4454848
嘉義市西區光路里 23.460092 120.440804
嘉義市西區翠岱里 23.4692651